In [1]:
#feature extraction from previous segmented images 
#to extract features for each image in the dataset such as texture, color, shape, size, etc
#and store them in a file
import cv2
import os
import numpy as np
from skimage.feature import hog
from skimage import data, exposure
import pywt
from imutils import face_utils
import cv2
import dlib

In [ ]:
#Feature Extractor using wavelets
class feature_extract:
    def __init__(self, wavelet= 'haar', level=1):
        self.wavelet = wavelet
        self.level = level

    def  extract(self, image):
        #convert image to float and scale to [0,1]
        image = image.astype('float32')/255

        #apply wavelet transform
        coeffs = pywt.wavedec2(image, self.wavelet, level=self.level)
        #extract features
        features = []
        for i, coeffs in enumerate(coeffs):
            if i == 0:
                #approximation coefficients
                features += self._extract_coeff_features(coeffs, 'A')
            
            else:
                #deatil coeeficients
                features += self._extract_coeff_features(coeffs[0], 'H')
                features += self._extract_coeff_features(coeffs[1], 'V')
                features += self._extract_coeff_features(coeffs[2], 'D')

        return features
    
    def _extract_coeff_features(self, coeff, label):
        #calculate statistics for wavelet coefficients
        features = []
        features.append(np.mean(coeff))
        features.append(np.std(coeff))
        features.append(np.median(coeff))
        features.append(np.max(coeff))
        features.append(np.min(coeff))

        # Add label to feature names
        features = [f'{label}_{f}' for f in features]

        return features
    

In [ ]:
#feature extraction using shape based

class shape_features:
    def __init__(self, image):
        self.image = image

    def extract_features(self, image_path):
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # detect and extract features
        detector = dlib.get_frontal_face_detector()
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
        faces = detector(gray)
        for face in faces:
            shape = predictor(gray, face)
            shape = face_utils.shape_to_np(shape)
            # loop over the (x, y)-coordinates for the facial landmarks
            # and draw them on the image
            for (x, y) in shape:
                cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
        # show the output image with the face detections + facial landmarks
        cv2.imshow("Output", image)
        cv2.waitKey(0)